In [1]:
import torch
from agents.bigtwo156 import Agent156
from agents.bigtwo376 import Agent376
from env.game import Bigtwo
from agents.random import RandomAgent

torch.manual_seed(0)
torch.cuda.manual_seed(0)
game = Bigtwo(42)

agents = [
    Agent376("cuda:0"),
    Agent376("cuda:0"),
    Agent376("cuda:0"),
    Agent376("cuda:0"),
]

In [2]:
for j in range(1000):
    winners = [0, 0, 0, 0]
    losses = [0, 0, 0, 0]
    for i in range(100):
        game.reset()
        while game.winner == None:
            agents[game.player_to_act].act(game)
        for index, agent in enumerate(agents):
            agent.rewards.append(1 if game.winner == index else -0.33)
        winners[game.winner] += 1

    for index, agent in enumerate(agents):
        loss = agent.learn()
        losses[index] = loss

    if j % 200 == 0:
        for index, agent in enumerate(agents):
            agent.save(index)
    print(winners)
    print(losses)

Model and optimizer saved as checkpoints/bigtwo376-0_20240923_145804.pth
Model and optimizer saved as checkpoints/bigtwo156-2_20240923_145804.pth
[14, 47, 6, 33]
[0.23307029902935028, 0, 0.10406450927257538, 0]
[1, 41, 3, 55]
[0.024257250130176544, 0, 0.05829351767897606, 0]
[4, 44, 3, 49]
[0.07122121006250381, 0, 0.05265111103653908, 0]
[2, 48, 2, 48]
[0.039825569838285446, 0, 0.03980705887079239, 0]
[3, 51, 3, 43]
[0.0554499477148056, 0, 0.05649690330028534, 0]
[3, 49, 2, 46]
[0.05597206577658653, 0, 0.038462527096271515, 0]
[6, 48, 3, 43]
[0.10143838077783585, 0, 0.053155966103076935, 0]
[26, 34, 3, 37]
[0.34836113452911377, 0, 0.054448094218969345, 0]
[18, 43, 1, 38]
[0.26274919509887695, 0, 0.02235298417508602, 0]
[21, 40, 1, 38]
[0.2811417281627655, 0, 0.02144668810069561, 0]
[14, 43, 2, 41]
[0.2225583791732788, 0, 0.03824325650930405, 0]
[31, 46, 0, 23]
[0.3765069544315338, 0, 0.00344003620557487, 0]
[54, 21, 0, 25]
[0.4303903579711914, 0, 0.001586686703376472, 0]
[61, 16, 0, 23

KeyboardInterrupt: 

In [33]:
agents[0].save('initial')

Model and optimizer saved as checkpoints/bigtwo376-initial_20240923_150604.pth


In [13]:
from agents.random import RandomAgent


players = [
    agents[0],
    RandomAgent(),
    RandomAgent(),
    RandomAgent(),
]

for _ in range(5):
    winners = [0, 0, 0, 0]
    losses = [0, 0, 0, 0]
    for i in range(500):
        game.reset()
        while game.winner == None:
            players[game.player_to_act].act(game)
        winners[game.winner] += 1
    print(winners)

[209, 19, 253, 19]
[243, 12, 215, 30]
[216, 18, 251, 15]
[238, 24, 220, 18]
[238, 16, 222, 24]
